In [1]:
%pip install python-dotenv

from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install langchain-community langchain-chroma langchain-openai bs4

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached asgiref-3.8.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached mypy_ext

In [3]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [7]:
loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("editedContent")
        )
    )
)
docs = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=api_key),
)

In [9]:
retriever = vectorstore.as_retriever()

user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

/opt/anaconda3/envs/myvenv/lib/python3.11/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘. \nContext: 배경을 가진 학생들이 함께 성장할 수 있는 환경이 조성됩니다. 글로벌 사회에서 중요한 소통 능력을 기를 수 있는 문화의 장으로 쓰일 수 있겠죠!사용한 기술 스택-FE(프론트엔드): Spring Boot, MySQL, Spring WebSocket-BE(백엔드): React Native, TanStack Query, Axios코딩 공모전 수상작은 대학생들의 팀프로젝트를 통해 만들어진 웹/앱 서비스입니다. 캠퍼스에서의 문제를 해결하자는 참가자들의 아이디어에서 시작되었죠. 누구나 세상에 선보이고 싶은 나만의 아이디어와 기초 코딩 기술만 활용한다면, 얼마든지 서비스를 만들 수 있습니다. 스파르타코딩클럽의 내일배움캠프에서는 비전공, 초보자도 웹/앱 개발자로 거듭날 수 있는 다양한 트랙이 준비돼 있습니다. 나만의 아이디어를 세상에 선보이고 싶은 누구나에게 열려 있으니 주저말고 도전해 보세요.💡<All-in> 코딩 공모전에서 만든 다양한 서비스를 만나보고 싶다면?다양한 서비스와 기발한 아이디어가 모인 곳에 초대합니다. 참가자들의 문제 해결방법이 궁금하시다면 지금 바로 ‘All-in 공모전’에서 만나보세요!👉🏻\xa0공모전 결과물 보러가기누구나 큰일 낼 수 있어스파르타코딩클럽글 | 신수지 팀스파르타 에디터\n\n코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀

In [10]:
response = llm.invoke(user_prompt)
print(response.content)

ALL-in 코딩 공모전의 수상작들은 대학생들이 캠퍼스에서 겪는 문제를 해결하기 위해 아이디어를 바탕으로 만든 웹/앱 서비스입니다. 주요 수상작으로는 언어 학습을 돕는 'Lexi Note', 동아리 정보를 제공하는 'Crewing', 그리고 실시간 소통을 지원하는 '에코 클래스룸' 등이 있습니다. 이러한 프로젝트들은 다양한 기술 스택을 활용하여 효율성을 높이고 참가자들의 창의력을 발휘하게 했습니다.
